# How to onboard a new Group/Team to Operate First
This guide explains how you can onboard a new group to the Operate First clusters. 

:::{seealso}

For more information about the Operate First see [Getting started with Operate First](references:operate-first-start)
:::

## Prerequisites
- There are no prerequisites.


## Outcomes

- [ ] A pull request against the `operate-first/apps` repository.

The PR enables Operate First to:

- [ ] Create a group on the Operate First fleet of clusters.
- [ ] Add desired users as to the created group.

## Introduction

All manifests for all the workloads owned by Operate First Ops team are maintained in the `operate-first/apps` repository following the [Kustomize best practices](https://kubernetes.io/docs/tasks/manage-kubernetes-objects/kustomization/).

The `cluster-scope` folder in this repo stores all privileged resources that are usually not allowed to be deployed by regular project admin and requires elevated access like cluster-admin role.

If you want to know more about the overall design please consult Operate First's [Architectural Decision Records (ADR) archive](https://www.operate-first.cloud/blueprints/blueprint/#architectural-decisions).


## Steps

### 1. Define important variables

In this guide we will use a couple of facts about the project. To make it easier to follow this guide, let's define these values beforehand.

In [1]:
import json

GITHUB_USERNAME = "4n4nd" # your github username

GROUP_NAME="demo-group"   # your desired team name

MEMBERS_LIST = [GITHUB_USERNAME,] # list of LOWERCASE github usernames of the team members

TEAM_MEMBERS=json.dumps([u.lower() for u in sorted(MEMBERS_LIST)]).replace("\"", "\\\"")


### 2. Fork and clone the apps repository

Please fork/clone the [operate-first/apps](https://github.com/operate-first/apps) repository. We’ll be working within this repository only.

1. Go to [operate-first/apps](https://github.com/operate-first/apps).
2. Click on a fork button.
3. When a fork is created click on the code button and copy an address of your forked repository.
4. Run following command using copied address:

In [2]:
!git clone https://github.com/{GITHUB_USERNAME}/apps.git
%cd apps

Cloning into 'apps'...
remote: Enumerating objects: 15289, done.
remote: Counting objects: 100% (1929/1929), done.
remote: Compressing objects: 100% (564/564), done.
remote: Total 15289 (delta 1408), reused 1675 (delta 1340), pack-reused 13360
Receiving objects: 100% (15289/15289), 5.34 MiB | 7.95 MiB/s, done.
Resolving deltas: 100% (7645/7645), done.
/home/anand/Documents/4n4nd/hitchhikers-guide/pages/apps


### 3. Creating group
####  Base resources


To create necessary resources we use opfcli which creates the following folders with files:

- A blank user group for the `$GROUP_NAME` if it does not exist yet in the `cluster-scope/base/user.openshift.io/groups/$GROUP_NAME`.

In [3]:
!opfcli create-group {GROUP_NAME}

INFO[0000] writing group definition to /home/anand/Documents/4n4nd/hitchhikers-guide/pages/apps/cluster-scope/base/user.openshift.io/groups/demo-group 


### 4. Populate your team
By executing the next cell, you will add the users listed in `$MEMBERS_LIST` to the OpenShift group `$GROUP_NAME`.

In [4]:
!yq e -i ".users = {TEAM_MEMBERS}" cluster-scope/base/user.openshift.io/groups/{GROUP_NAME}/group.yaml
!cat cluster-scope/base/user.openshift.io/groups/{GROUP_NAME}/group.yaml

apiVersion: user.openshift.io/v1
kind: Group
metadata:
  name: demo-group
users: [4n4nd]


### 5. Add new team to common kustomization

This will apply the newly created group manifest to all the exisitng Operate First clusters.

In [5]:
!cd cluster-scope/overlays/prod/common && kustomize edit add resource ../../../base/user.openshift.io/groups/{GROUP_NAME}

## Finalize
Please stage your changes and send them as a PR against the [operate-first/apps](https://github.com/operate-first/apps) repository. 

:::{note}
Make sure that following files/ have been modified/added:
- [x] `cluster-scope/base/user.openshift.io/groups/$GROUP_NAME/group.yaml`
- [x] `cluster-scope/base/user.openshift.io/groups/$GROUP_NAME/kustomization.yaml`
- [x] `cluster-scope/overlays/prod/common/kustomization.yaml`
:::

Once the PR is merged, all the desired changes should be applied.

In [7]:
!git status
!git add .
!git commit -m "feat(onboarding): Add group {GROUP_NAME}"
!git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   cluster-scope/overlays/prod/common/kustomization.yaml

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cluster-scope/base/user.openshift.io/groups/demo-group/

no changes added to commit (use "git add" and/or "git commit -a")
[master d3cbed4] feat(onboarding): Add group demo-group
 3 files changed, 58 insertions(+), 48 deletions(-)
 create mode 100644 cluster-scope/base/user.openshift.io/groups/demo-group/group.yaml
 create mode 100644 cluster-scope/base/user.openshift.io/groups/demo-group/kustomization.yaml
 rewrite cluster-scope/overlays/prod/common/kustomization.yaml (97%)
Enumerating objects: 21, done.
Counting objects: 100% (21/21), done.
Delta compression using up to 12 threads
Compressing objects: 100% (12